**Project in course Building Innovative Systems**

Prediction in time series dataset

Submitted by :

Shantanu Dev

101917052

CSE-3

**SECTION 1: READING DATA**

In [6]:
import pandas as pd
import os
import numpy as np
import math
df = pd.read_excel('./DATASET.xlsx')
df


/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Unnamed: 0,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
0,Sec_1,1,324.0,354.5,17.0,0.0,3,117,2600,400,0.0,0.0,0.0,0,1.520
1,NaN,2,324.0,161.1,18.2,0.0,4,106,5950,1190,0.0,3.4,0.0,0,1.620
2,NaN,3,324.0,170.7,18.5,0.0,4,110,5950,1190,0.0,21.3,0.0,4,1.680
3,NaN,4,324.0,223.9,18.9,0.0,3,110,6150,1340,0.0,21.3,0.0,5,1.780
4,NaN,5,324.0,228.2,18.4,0.0,2,113,6340,1450,0.0,23.9,0.0,7,1.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,NaN,6,454.0,931.8,13.6,125.0,13,116,4150,830,15.1,1.8,1.0,15,1.681
1005,NaN,7,454.0,703.0,14.5,17.0,13,118,4150,830,24.6,7.9,2.0,15,1.838
1006,NaN,8,454.0,1124.1,14.9,45.0,18,117,4200,840,119.0,15.7,2.0,16,1.862
1007,NaN,9,454.0,1462.3,13.2,62.0,27,117,4400,720,138.2,30.5,6.0,18,1.883


In [7]:
df.dtypes

Unnamed: 0     object
year            int64
Para-1        float64
Para-2        float64
Para-3        float64
Para-4        float64
Para-5          int64
Para-6          int64
Para-7          int64
Para-8          int64
Para-9        float64
Para-10       float64
Para-11       float64
Para-12         int64
Para-13       float64
dtype: object

**SECTION 2: REMOVING IRREVELANT ENTRIES**

***1.) REMOVED SECTIONS WHERE DATA FOR 10TH ROW IS ABSENT***

***2.) CREATING SEPARATE SECTION FILES OF VALID SECTIONS***

In [8]:
path = './samples/'
isExist = os.path.exists(path)
if not isExist: 
  os.makedirs(path)

In [9]:
df2 = pd.DataFrame()
s_no = ""
i = 0
new_df = pd.DataFrame()
while i < len(df):
    j = 0
    if type(df.iloc[i][0])==str:
        s_no = df.iloc[i][0]
        j+=1
    j+=i
    exist = False
    while j < len(df) and type(df.iloc[j][0])!=str:
        if df.iloc[j][1] == 10:
            exist = True
        j += 1
    
    if(exist):
        df2 = df2.append(df.loc[i:j-1,:])
        new_df = new_df.append(df.loc[i:j-1,:])
        new_df.to_csv('./samples/'+s_no+'.csv',index=False)
        new_df = new_df.iloc[0:0]
    i = j

df2.reset_index(inplace=True,drop=True)


**SECTION 3: STORING THE TARGET DATA**


In [10]:
verification = df2[df2.year == 10]
verification = verification.reset_index()
verification.drop(columns=['index', 'Unnamed: 0'], inplace=True)
verification

,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
0,10,324.0,53.6,19.0,0.0,2,105,7567,1885,16.8,105.0,65.0,9,1.960
1,10,327.7,62.0,22.7,0.0,2,102,30525,10234,76.5,44.3,5.0,9,0.795
2,10,487.7,93.2,22.5,0.0,4,110,2140,1270,78.5,0.0,0.0,16,1.432
3,10,543.6,93.2,22.5,0.0,4,110,2140,1270,28.5,60.5,67.0,11,2.004
4,10,434.4,149.0,11.7,72.0,3,110,13142,6225,0.0,276.2,34.0,6,1.792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,514.0,1226.4,10.1,203.6,16,114,7672,303,219.7,206.2,10.0,8,1.607
96,10,901.0,1531.0,11.1,34.9,40,100,15150,1077,5.9,74.5,33.0,13,1.087
97,10,1181.0,1364.9,9.2,274.3,24,112,12500,880,131.4,72.4,18.0,12,0.996
98,10,662.0,1110.7,12.8,33.2,27,117,3324,1356,193.7,305.6,180.0,12,2.212


**SECTION 4: ANALYSIS PHASE**

In [11]:
#importing libraries
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_squared_error
import glob
from sklearn.decomposition import PCA

In [13]:
path = "./samples/*.csv"
for fname in glob.glob(path):
    tdf = pd.read_csv(fname)
    tdf.drop(columns=['Unnamed: 0'],inplace=True)
    sec_no = fname.split('/')[2].split('.')[0]

    rmse_df = pd.DataFrame(columns=['Output para'])

    for i in range(9,14):
        folder_path = './comparisons/'
        isExist = os.path.exists(folder_path)
        if not isExist:
            os.makedirs(folder_path)

        x = tdf.drop(columns=['Para-9', 'Para-10',	'Para-11',	'Para-12',	'Para-13'], axis=1)
        y = tdf['Para-'+str(i)]
        x_train, x_test = x.iloc[:-1, :], x.iloc[-1:, :]
        y_train, y_test = y.iloc[:-1], y.iloc[-1:]

        model = XGBRegressor(objective='reg:squarederror',random_state=51,verbosity = 0)
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        rmse = np.round(math.sqrt(mean_squared_error(y_test, predictions)), 3)
        actual = y_test.values
        rmse_df.loc[0, 'Para-'+str(i)] = actual[0]
        rmse_df.loc[1, 'Para-'+str(i)] = np.round(predictions[0], 3)
        rmse_df.loc[2, 'Para-'+str(i)] = rmse
    
    rmse_df.to_csv(folder_path+'/'+sec_no+'.csv', index=False)

**SECTION 5: PLOTTING DATA and CALCUATING ERRORS**

In [14]:
rmse_collection = []

folder_path = './graphs/'
isExist = os.path.exists(folder_path)
if not isExist:
  os.makedirs(folder_path)

path = "./comparisons/*.csv"
for fname in glob.glob(path):
  rmse_df = pd.read_csv(fname)
  sec_no = fname.split('/')[2].split('.')[0]

  mean_rmse = np.round(rmse_df.iloc[2, :].mean(axis=0), 3)
  rmse_collection.append(mean_rmse)
  line_1 = rmse_df.iloc[0, :]
  line_2 = rmse_df.iloc[1, :]
  fig, ax = plt.subplots()
  ax.plot(line_1, color = 'green', label = 'Actual Data')
  ax.plot(line_2, color = 'red', label = 'Predicted Data')
  plt.title("RMSE Graph: "+str(mean_rmse))
  ax.legend()
  plt.savefig("./graphs/"+str(sec_no)+".png")
  plt.close()

In [16]:
f_rmse = sum(rmse_collection)/len(rmse_collection)
print("RMSE: ", np.round(f_rmse, 3))

RMSE:  9.355
